In [2]:
!pip install python-cmr==0.4.1
!pip install rioxarray==0.3.1
;

''

In [2]:
# https://nsidc.org/data/MOD10A1/versions/6
# https://github.com/jddeal/python-cmr

In [3]:
import os
import sys
import time
import math
import io
import rioxarray
import datetime
import base64
#import requests
import yaml
from urllib.request import urlopen, Request, build_opener, HTTPCookieProcessor

from cmr import GranuleQuery

In [4]:
#load_dotenv(envpath)
envvars = open('sat.yml' ,'r')
secrets = yaml.load(envvars, Loader=yaml.FullLoader)
envvars.close()
earthdata_user=secrets['EARTHDATA_USER']
earthdata_pass=secrets['EARTHDATA_PASS']

In [5]:
product = {
                'name': 'daily',
                'product': 'MOD10A1',
                'version': 6,
                'date_span': 20
            }

In [6]:
query_handler = GranuleQuery()

In [7]:
lower_left_lon = -141.899414
lower_left_lat = 47.783635
upper_right_lon = -112.104492
upper_right_lat = 60.866312
bbox = [lower_left_lon, lower_left_lat, upper_right_lon, upper_right_lat]

In [8]:
query_handler.bounding_box(*bbox)
query_handler.short_name(product['product'])
query_handler.version(product['version'])
end_date = datetime.datetime(2021, 2, 25).date()
start_date = end_date - datetime.timedelta(product['date_span'])
query_handler.temporal(f"{start_date}T00:00:00Z", f"{end_date}T23:59:59Z")

In [9]:
results = query_handler.get_all()
print(len(results))

273


In [10]:
url = results[0]['links'][0]['href']
url

'https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.006/2021.02.05/MOD10A1.A2021036.h08v04.006.2021038043413.hdf'

In [11]:
try:
    os.makedirs('modis')
except:
    pass

In [12]:
def cmr_read_in_chunks(file_object, chunk_size=1024 * 1024):
    """Read a file in chunks using a generator. Default chunk size: 1Mb."""
    while True:
        data = file_object.read(chunk_size)
        if not data:
            break
        yield data

In [13]:
"""stream = session.get(
            url,
            auth=auth,
            allow_redirects=True,
            stream=True
           )""";

In [13]:
def output_progress(count, total, status='', bar_len=60):
    if total <= 0:
        return
    fraction = min(max(count / float(total), 0), 1)
    filled_len = int(round(bar_len * fraction))
    percents = int(round(100.0 * fraction))
    bar = '=' * filled_len + ' ' * (bar_len - filled_len)
    fmt = '  [{0}] {1:3d}%  {2}   '.format(bar, percents, status)
    print('\b' * (len(fmt) + 4), end='')  # clears the line
    sys.stdout.write(fmt)
    sys.stdout.flush()
    
def get_speed(time_elapsed, chunk_size):
    if time_elapsed <= 0:
        return ''
    speed = chunk_size / time_elapsed
    if speed <= 0:
        speed = 1
    size_name = ('', 'k', 'M', 'G', 'T', 'P', 'E', 'Z', 'Y')
    i = int(math.floor(math.log(speed, 1000)))
    p = math.pow(1000, i)
    return '{0:.1f}{1}B/s'.format(speed / p, size_name[i])

In [14]:
def download(url):
    auth = (earthdata_user, earthdata_pass)
    #session = requests.Session()
    opener = build_opener(HTTPCookieProcessor())
    req = Request(url)
    credentials = '{0}:{1}'.format(earthdata_user, earthdata_pass)
    credentials = base64.b64encode(credentials.encode('ascii')).decode('ascii')
    req.add_header('Authorization', 'Basic {0}'.format(credentials))
    response = opener.open(req)
    length = int(response.headers['content-length'])
    chunk_size = min(max(length, 1), 1024 * 1024)
    max_chunks = int(math.ceil(length / chunk_size))
    count = 0
    time_initial = time.time()
    filename = url.split('/')[-1]
    date = url.split('/')[-2]
    out_pth = os.path.join('modis',date,filename)
    try:
        os.makedirs(os.path.split(out_pth)[0])
    except:
        pass
    with open(out_pth, 'wb') as f:
        for chunk in cmr_read_in_chunks(response, chunk_size=chunk_size):
            f.write(chunk)
            count = count + 1
            time_elapsed = time.time() - time_initial
            download_speed = get_speed(time_elapsed, count * chunk_size)
            output_progress(count, max_chunks, status=download_speed)


In [15]:
for res in results:
    url = res['links'][0]['href']
    print(url)
    download(url)

https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.006/2021.02.05/MOD10A1.A2021036.h08v04.006.2021038043413.hdf
  [==============================                              ]  50%  5.0MB/s    [============================================================] 100%  7.7MB/s   https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.006/2021.02.05/MOD10A1.A2021036.h10v02.006.2021038043505.hdf
  [====================                                        ]  33%  5.0MB/s    [========================================                    ]  67%  6.8MB/s    [============================================================] 100%  9.7MB/s   https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.006/2021.02.05/MOD10A1.A2021036.h09v03.006.2021038043526.hdf
  [==============================                              ]  50%  5.0MB/s    [============================================================] 100%  9.2MB/s   https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.006/2021.02.05/MOD10A1.A2021036.h10v04.006.2021038043526.hd